# Solution 3: Computing path lengths of taxi trips

This notebook is a solution to the problem posed in [exercises/exercise-3.ipynb](../exercises/exercise-3.ipynb).

Don't peek at this until you've tried to solve it!

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import awkward._v2 as ak

## Reading the dataset

For brevity, let's load the dataset with only the columns we need.

In [ ]:
taxi = ak.from_parquet(
    "https://zenodo.org/records/14537442/files/chicago-taxi.parquet",
    columns=["trip.km", "trip.begin.l*", "trip.path.*"],
    row_groups=[0],
)
taxi.show(type=True)

## Longitude, latitude → kilometers

Reproducing the conversion from longitude, latitude differences to absolute positions to kilometers.

In [ ]:
all_longitudes = taxi.trip.begin.lon + taxi.trip.path.londiff
all_latitudes = taxi.trip.begin.lat + taxi.trip.path.latdiff

all_longitudes.show()

In [ ]:
LONGITUDE, LATITUDE = -87.66178320769112, 41.896777262111726   # mean position (center of town)
LON_TO_KM, LAT_TO_KM = 82.98452409203695, 111.07127961503745   # lon, lat → km conversion factors

In [ ]:
km_east, km_north = (
    (all_longitudes - LONGITUDE) * LON_TO_KM,
    (all_latitudes - LATITUDE) * LAT_TO_KM,
)
km_east, km_north

## Single path example for comparison

This imperative `for` loop calculation on a single path was given in the problem statement.

I'm repeating it here so that we can compare it with the final answer (always a good idea!).

In [ ]:
example_east, example_north = (
    km_east[75, -2],
    km_north[75, -2],
)
example_east, example_north

In [ ]:
length = 0
for i in range(len(example_east) - 1):
    Δx = example_east[i + 1] - example_east[i]
    Δy = example_north[i + 1] - example_north[i]
    Δr = np.sqrt(Δx**2 + Δy**2)
    length += Δr

length

## THE SOLUTION

Here is a one-line calculation of all the distances of all the paths, maintaining path structure.

In [ ]:
distances = ak.sum(np.sqrt((km_east[:, :, 1:] - km_east[:, :, :-1])**2 + (km_north[:, :, 1:] - km_north[:, :, :-1])**2), axis=-1)

It has the right type: only a single `var` (same as `trips.km`).

In [ ]:
distances.type.show()

Here are the values that were provided for comparison, to help you know if you got it right.

In [ ]:
distances.show()

And here's the one distance that had been calculated with a `for` loop. It agrees.

In [ ]:
distances[75, -2]

## As the crow flies solution

There are two ways we could do this, but let's use the padding method.

The answer to

> How would you apply this technique to _deeply_ nested lists?

for both techniques is to use `axis=-1`.

In [ ]:
pad_km_east = ak.fill_none(ak.pad_none(km_east, 2, axis=-1), 0, axis=-1)
pad_km_north = ak.fill_none(ak.pad_none(km_north, 2, axis=-1), 0, axis=-1)

The "as the crow flies" solution is similar to the main problem, except that there's no `ak.sum` over segments. It's a single segment from item `-1` to item `0`.

In [ ]:
crow_flies = np.sqrt((pad_km_east[:, :, -1] - pad_km_east[:, :, 0])**2 + (pad_km_north[:, :, -1] - pad_km_north[:, :, 0])**2)
crow_flies.show()

## Plotting differences with the `trip.km` field

The plotting code was given, including the hint of filling missing values with `0` (so that `distances` and `taxi.trip.km` align) and flattening.

Here are the full distances on the horizontal axis and `trip.km` on the vertical axis.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

ax.hist2d(
    ak.flatten(ak.fill_none(distances, 0)),    # None → 0 on the plot
    ak.flatten(ak.fill_none(taxi.trip.km, 0)), # None → 0 on the plot
    bins=(200, 200),
    range=((0, 50), (0, 50)),
    norm=matplotlib.colors.LogNorm(),
);
ax.plot([0, 50], [0, 50], c="orange", lw=3);

Here are "as the crow flies" distances on the horizontal axis and `trip.km` on the vertical axis.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

ax.hist2d(
    ak.flatten(ak.fill_none(crow_flies, 0)),   # None → 0 on the plot
    ak.flatten(ak.fill_none(taxi.trip.km, 0)), # None → 0 on the plot
    bins=(200, 200),
    range=((0, 50), (0, 50)),
    norm=matplotlib.colors.LogNorm(),
);
ax.plot([0, 50], [0, 50], c="orange", lw=3);

We should not be surprised that the full path calculations are a better match to reported `trip.km` measurements.

The "as the crow flies" are more fictional: cars don't drive that way.

<img src="../img/as-the-crow-flies.svg" width="500">

However, there are some interesting features on this plot that _I do not understand._

   * There's an island of `trip.km` $\approx$ 29 km with calculated distances $\approx$ 3 km.
   * There's a secondary trend of `trip.km` $\approx$ `distance / 15` along the bottom of each plot. What could that be?

Fun with real data!